In [3]:
from pybis import Openbis
import json
import os
import ipywidgets as widgets
from IPython.display import display, Javascript, HTML, clear_output
from ipyfilechooser import FileChooser
from datetime import datetime

# TODO: Try to get a better approach
import sys
sys.path.append('/home/jovyan/aiida-openbis/Notebooks/importer')
import nanonis_importer

In [6]:
# Global variables
OPENBIS_SESSION = None
SESSION_DATA = None
RAW_MATERIALS_TYPES = ["CRYSTAL", "WAFER_SUBSTRATE", "2D_LAYERED_MATERIAL", "MOLECULE"]
PROCESS_SAMPLE_TYPES = ["SPUTTERING", "ANNEALING", "DEPOSITION"]
MEASUREMENT_FILE_EXTENSIONS = [".sxm", ".dat"]

In [ ]:
# Widgets
samples_dropdown = widgets.Dropdown(description='Sample', disabled=False, layout = widgets.Layout(width = '400px'))
samples_dropdown.style = {'description_width': '110px'}
sample_details_textbox = widgets.Textarea(value = '', description = '', disabled = True, layout = widgets.Layout(width = '600px', height = '300px'))
sample_metadata_boxes = widgets.HBox([samples_dropdown, sample_details_textbox])

instruments_dropdown = widgets.Dropdown(description='Instrument', disabled=False, layout = widgets.Layout(width = '993px'))
instruments_dropdown.style = {'description_width': '110px'}

measurements_files_import = widgets.FileUpload(accept = '.sxm, .dat', multiple = True)

upload_measurements_button = widgets.Button(description = '', disabled = False, button_style = '', tooltip = 'Save measurements', icon = 'save', layout = widgets.Layout(width = '100px', height = '50px'))
quit_button = widgets.Button(description = '', disabled = False, button_style = '', tooltip = 'Main menu', icon = 'home', layout = widgets.Layout(width = '100px', height = '50px'))

bottom_buttons_hbox = widgets.HBox([upload_measurements_button, quit_button])

increase_buttons_size = HTML("""
<style>
    .fa-save {
        font-size: 2em !important; /* Increase icon size */
    }
    .fa-home {
        font-size: 2em !important; /* Increase icon size */
    }
</style>
""")

folder_selector = FileChooser('.', select_default=True, use_dir_icons=True)
folder_selector.show_only_dirs = True  # Set to True to display only folders

In [7]:
# Functions
def read_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

# Function to close the notebook page
def close_notebook(b):
    display(Javascript(f'window.open("home.ipynb", "_blank")'))
    display(Javascript('window.close()'))

def load_sample_metadata(change):
    global OPENBIS_SESSION
    if samples_dropdown.value is None:
        sample_details_textbox.value = ''
    else:
        sample_object = OPENBIS_SESSION.get_object(samples_dropdown.value)
        sample_parents_metadata = []
        sample_parents_metadata = get_parents_recursive(sample_object, sample_parents_metadata)
        
        last_sample_process = None
        sample_processes_strings = []
        sample_materials_strings = []
        number_parents = len(sample_parents_metadata)
        parent_idx = 0
        while parent_idx < number_parents:
            parent_metadata = sample_parents_metadata[parent_idx]
            if parent_metadata[0] == "DEPOSITION":
                next_parent_metadata = sample_parents_metadata[parent_idx + 1]
                sample_metadata_string = f"> {parent_metadata[0]} ({parent_metadata[3]}, {parent_metadata[1]}, {parent_metadata[2]}) [{next_parent_metadata[0]} ({next_parent_metadata[3]}, {next_parent_metadata[1]}, {next_parent_metadata[4]})]"
                parent_idx += 1
            else:
                sample_metadata_string = f"> {parent_metadata[0]} ({parent_metadata[3]}, {parent_metadata[1]}, {parent_metadata[2]})"
            
            if parent_metadata[0] in PROCESS_SAMPLE_TYPES:
                sample_processes_strings.append(sample_metadata_string)
                # Get the last sample preparation method performed on the sample in order to search the correct instrument where the sample is being used.
                if last_sample_process is None:
                    last_sample_process = parent_metadata[1]
            else:
                sample_materials_strings.append(sample_metadata_string)
                
            parent_idx += 1
        
        sample_metadata_string = f"PermId: {sample_object.attrs.permId}\nMaterial:"
        
        for material_string in sample_materials_strings:
            sample_metadata_string = f"{sample_metadata_string}\n{material_string}"
            
        sample_metadata_string = f"{sample_metadata_string}\nProcesses:"
        for process_string in sample_processes_strings:
            sample_metadata_string = f"{sample_metadata_string}\n{process_string}"

        if last_sample_process is not None:
            last_sample_process_object = OPENBIS_SESSION.get_object(last_sample_process)
            
            # Automatically select the instrument used in the last sample process task
            last_sample_process_object_parents = last_sample_process_object.parents
            for parent in last_sample_process_object_parents:
                parent_object = OPENBIS_SESSION.get_object(parent)
                if parent_object.type == "INSTRUMENT":
                    instruments_dropdown.value = parent_object.permId
        
        sample_details_textbox.value = sample_metadata_string

def get_parents_recursive(object, object_parents_metadata):
    if object.attrs.type in PROCESS_SAMPLE_TYPES or object.attrs.type in RAW_MATERIALS_TYPES:
        object_parents_metadata.append([object.attrs.type, object.attrs.permId, object.attrs.registrationDate, 
                                        object.props['$name'], object.props['sum_formula']])
    object_parents = object.parents
    for parent in object_parents:
        parent_object = OPENBIS_SESSION.get_object(parent)
        get_parents_recursive(parent_object, object_parents_metadata)
    return object_parents_metadata

def upload_measurements_to_openbis(b):
    measurements_collection = None
    sample_object = OPENBIS_SESSION.get_object(samples_dropdown.value)
    for child_id in sample_object.children:
        child_object = OPENBIS_SESSION.get_object(child_id)
        if child_object.type == "1D_MEASUREMENT" or child_object.type == "2D_MEASUREMENT":
            measurements_collection = child_object.collection
            break
    
    sample_project = None
    for parent_id in sample_object.parents:
        parent_object = OPENBIS_SESSION.get_object(parent_id)
        if parent_object.type in PROCESS_SAMPLE_TYPES:
            sample_project = parent_object.project
            break
    
    importer_script = "/home/jovyan/aiida-openbis/Notebooks/importer/nanonis_importer.py"
    data_folder = folder_selector.selected_path
    
    correct_folder = False
    data_files = os.listdir(data_folder)
    for filename in data_files:
        filename_extension = filename.split('.')[-1]
        if f".{filename_extension}" in MEASUREMENT_FILE_EXTENSIONS:
            correct_folder = True
        else:
            correct_folder = False
            break
    
    if correct_folder:
        if measurements_collection:
            measurements_collection = f"{sample_project.identifier}/{measurements_collection}"
            nanonis_importer.upload_measurements_into_openbis(SESSION_DATA['url'], data_folder, measurements_collection, sample_object.permId, instruments_dropdown.value)
            print("Upload successful!")
            
        elif sample_project:
            measurements_collection = OPENBIS_SESSION.new_collection(code = f"MEASUREMENTS_COLLECTION_{sample_object.code}", 
                                                                    type = "COLLECTION", 
                                                                    project = sample_project,
                                                                    props = {"$name": f"Measurements from Sample {sample_object.props['$name']}",
                                                                            "$default_collection_view": "IMAGING_GALLERY_VIEW"}
                                                                    )
            measurements_collection.save()
            nanonis_importer.upload_measurements_into_openbis(SESSION_DATA['url'], data_folder, measurements_collection.permId, sample_object.permId, instruments_dropdown.value)
            print("Upload successful!")
            
        else:
            print("Sample does not belong to any project yet.")
            
    else:
        print(f"Folder contains unrecognised files. Please remove files whose extension does not belong to the following list {MEASUREMENT_FILE_EXTENSIONS}.")

In [8]:
# Connect with openBIS
SESSION_DATA = read_json("token.json")
OPENBIS_SESSION = Openbis(SESSION_DATA["url"], verify_certificates = False)
OPENBIS_SESSION.set_token(SESSION_DATA["token"])

# Measurement Uploader

## Select sample

In [ ]:
samples = OPENBIS_SESSION.get_objects(type = "SAMPLE")
instruments = OPENBIS_SESSION.get_objects(type = "INSTRUMENT")

samples_names_permids = [(sample.props['$name'], sample.permId) for sample in samples]
samples_dates = [datetime.fromisoformat(sample.registrationDate) for sample in samples]
samples_names_permids = [x for _,x in sorted(zip(samples_dates, samples_names_permids), reverse = True)]
samples_names_permids.insert(0, ('Select an input sample...', None))
samples_dropdown.options = samples_names_permids

instruments_names_permids = [(f"{instrument.props['$name']} ({instrument.attrs.permId})", instrument.permId) for instrument in instruments]
instruments_names_permids.insert(0, ('Select an instrument...', None))
instruments_dropdown.options = instruments_names_permids
instruments_dropdown.options = instruments_names_permids

display(widgets.VBox([sample_metadata_boxes, instruments_dropdown]))
upload_measurements_button.on_click(upload_measurements_to_openbis)
quit_button.on_click(close_notebook)
samples_dropdown.observe(load_sample_metadata, names = 'value')

## Select measurements folders

In [ ]:
display(folder_selector)
display(bottom_buttons_hbox)
display(increase_buttons_size)